First I'll add the libraries and create the neccesary functions for this code to work properly

In [2]:
import pandas as pd
import numpy as np
import re
# Filter all warnings. 
import warnings
warnings.filterwarnings('ignore')

# Function to clean the names 
def Clean_names(Country_name): 
    # Search for opening bracket in the name followed by 
    # any characters repeated any number of times 
    if re.search('\(.*', Country_name): 
        # Extract the position of beginning of pattern 
        pos = re.search('\(.*', Country_name).start() 
        # return the cleaned name 
        return Country_name[:pos] 
    elif re.search('\d', Country_name): 
        # Extract the position of beginning of pattern 
        pos = re.search('\d', Country_name).start() 
        # return the cleaned name 
        return Country_name[:pos]
    else: 
        # if clean up needed return the same name 
        return Country_name 


Dataset importing, cleaning and such

In [12]:
# Importing the excel dataset and do a bit of cleaning to get rid of the useless columns and rows
Energy=pd.read_excel('assets/Energy Indicators.xls', header=16)
Energy=Energy[:228].drop([0]).drop([Energy.columns[0] , Energy.columns[1]] ,  axis='columns')

#Next, I'll change some of the labes for readability
Energy.rename(columns={'Unnamed: 2': 'Country', 'Renewable Electricity Production': '% Renewable'}, inplace=True)
#change some of the missing data to NaN values using lambda functions
Energy['Energy Supply']=Energy['Energy Supply'].apply(lambda x: np.nan if type(x)!=int else x)
Energy['Energy Supply per capita']=Energy['Energy Supply per capita'].apply(lambda x: np.nan if type(x)!=int else x)
#and then converting petabytes to gigabytes
Energy['Energy Supply']=Energy['Energy Supply']*1000000

#We apply the cleaning function so we only get the names, without special characters or numbers
Energy['Country'] = Energy['Country'].apply(Clean_names) 
#Then we rename some of the countries that will be used later on (I guess there's a better way, but idk how right now)  
Energy.set_index('Country', inplace=True)
Energy.rename(index={"Republic of Korea": "South Korea",
                    "United States of America": "United States",
                    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                    "China, Hong Kong Special Administrative Region": "Hong Kong"}, inplace=True)
#Energy.reset_index(inplace=True) #no need to do this because we will merge the different datasets using the index
#We load the GDP file and clean it a bit
GDP=pd.read_csv('assets/utf-8\'\'world_bank.csv', header=4)
GDP.rename(columns={"Country Name": "Country"}, inplace=True)
GDP.set_index('Country', inplace=True)
GDP.rename(index={"Korea, Rep.": "South Korea", 
                        "Iran, Islamic Rep.": "Iran",
                        "Hong Kong SAR, China": "Hong Kong"}, inplace=True)
#Finally, we load the ScimEn data

ScimEn=pd.read_excel('assets/utf-8\'\'scimagojr-3.xlsx')
ScimEn.set_index('Country', inplace=True)

,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,,
Aruba,ABW,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.467704e+09,NaN,NaN,NaN,NaN,NaN
Andorra,AND,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.018196e+09,4.021331e+09,3.675728e+09,3.535389e+09,3.346317e+09,3.185605e+09,3.129538e+09,3.127550e+09,NaN,NaN
Afghanistan,AFG,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.030523e+10,1.172119e+10,1.214448e+10,1.469733e+10,1.593680e+10,1.691113e+10,1.935220e+10,1.973134e+10,1.999032e+10,2.029415e+10
Angola,AGO,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.581103e+10,6.842044e+10,7.787420e+10,7.975320e+10,8.247091e+10,8.570262e+10,9.012096e+10,9.626143e+10,1.008863e+11,1.039106e+11
Albania,ALB,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.771760e+09,1.034829e+10,1.112752e+10,1.150029e+10,1.192695e+10,1.223109e+10,1.240477e+10,1.254247e+10,1.279331e+10,1.312082e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Yemen, Rep.",YEM,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.672565e+10,2.761787e+10,2.872656e+10,2.991436e+10,3.090675e+10,2.624342e+10,2.689160e+10,2.800914e+10,NaN,NaN
South Africa,ZAF,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,7.707081e+10,8.003449e+10,8.497844e+10,9.124438e+10,9.848904e+10,1.072507e+11,1.120109e+11,...,3.402852e+11,3.585261e+11,3.699668e+11,3.642764e+11,3.753494e+11,3.874074e+11,3.960071e+11,4.047682e+11,4.110369e+11,4.163117e+11
"Congo, Dem. Rep.",COD,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,1.508024e+10,1.344383e+10,1.629401e+10,1.714358e+10,1.672524e+10,1.689205e+10,1.803656e+10,...,1.650894e+10,1.754232e+10,1.863448e+10,1.916651e+10,2.052329e+10,2.193213e+10,2.350200e+10,2.550050e+10,2.778776e+10,2.970961e+10


Now, we merge those datasets into a new one, doing an intersection (inner merge) to see only the countries of interest

In [89]:
new_df=pd.merge(Energy, GDP.drop(GDP.iloc[:,3:49], axis = 1) , how='outer', left_index=True, right_index=True) #idk why, but if I do the inner merge I don't get the desired output, but this should be correct
new_df=pd.merge(new_df, ScimEn[:15], how='inner', left_index=True, right_index=True)

#new_df=new_df[['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
#this where the requested columns for the project, but I wanted to see the entire dataframe
new_df=new_df.reset_index().set_index('Rank').sort_index()

new_df




,Country,Energy Supply,Energy Supply per capita,% Renewable,Country Code,Indicator Name,Indicator Code,2006,2007,2008,...,2012,2013,2014,2015,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
Rank,,,,,,,,,,,,,,,,,,,,,
1,China,1.271910e+11,93.0,19.7549,CHN,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,3.992331e+12,4.559041e+12,4.997775e+12,...,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12,127050,126767,597237,411683,4.70,138
2,United States,9.083800e+10,286.0,11.571,USA,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,1.479230e+13,1.505540e+13,1.501149e+13,...,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13,96661,94747,792274,265436,8.20,230
3,Japan,1.898400e+10,149.0,10.2328,JPN,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,5.496542e+12,5.617036e+12,5.558527e+12,...,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12,30504,30287,223024,61554,7.31,134
4,United Kingdom,7.920000e+09,124.0,10.6005,GBR,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,2.419631e+12,2.482203e+12,2.470614e+12,...,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12,20944,20357,206091,37874,9.84,139
5,Russian Federation,3.070900e+10,214.0,17.2887,RUS,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,1.385793e+12,1.504071e+12,1.583004e+12,...,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12,18534,18301,34266,12422,1.85,57
6,Canada,1.043100e+10,296.0,61.9454,CAN,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,1.564469e+12,1.596740e+12,1.612713e+12,...,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12,17899,17620,215003,40930,12.01,149
7,Germany,1.326100e+10,165.0,17.9015,DEU,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,3.332891e+12,3.441561e+12,3.478809e+12,...,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12,17027,16831,140566,27426,8.26,126
8,India,3.319500e+10,26.0,14.9691,IND,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,1.265894e+12,1.374865e+12,1.428361e+12,...,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12,15005,14841,128763,37209,8.58,115
9,France,1.059700e+10,166.0,17.0203,FRA,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,2.607840e+12,2.669424e+12,2.674637e+12,...,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12,13153,12973,130632,28601,9.93,114
